**Table of contents**<a id='toc0_'></a>    
- [使用Foursquare数据](#toc1_)    
  - [下载](#toc1_1_)    
    - [下载参考](#toc1_1_1_)    
    - [AWS S3下载步骤（已验证）](#toc1_1_2_)    
    - [没有验证的方法](#toc1_1_3_)    
      - [方法2: 使用 boto3 脚本并启用多线程](#toc1_1_3_1_)    
      - [方法3: 使用 AWS S3 Transfer Acceleration](#toc1_1_3_2_)    
      - [方法4: 利用 EC2 实例中转](#toc1_1_3_3_)    
      - [最佳实践和优化建议](#toc1_1_3_4_)    
  - [使用](#toc1_2_)    
    - [使用参考](#toc1_2_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[使用Foursquare数据](#toc0_)

## <a id='toc1_1_'></a>[下载](#toc0_)

### <a id='toc1_1_1_'></a>[下载参考](#toc0_)

1. 信息来源：<https://mp.weixin.qq.com/s/PjvodYVRRKxOflUdeK2c8A>
2. 亚马逊安全凭证管理页面: <https://us-east-1.console.aws.amazon.com/iam/home?region=us-east-1#/security_credentials>
3. 具体数据说明: <https://docs.foursquare.com/data-products/docs/access-fsq-os-places>
    ```log
    Prerequisites
    In order to access Foursquare's Open Source Places data, you will need the following:

    S3 paths:
    - Places - s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/parquet
    - Categories - s3://fsq-os-places-us-east-1/release/dt=2024-12-03/categories/parquet
    AWS CLI or SDK’s to download the parquet files and use in your local environment
    If using DuckDb, please refer to the Using DuckDb section for instructions on how to create remote/local tables for use.
    ```

### <a id='toc1_1_2_'></a>[AWS S3下载步骤（已验证）](#toc0_)

由ChatGPT提供。

方法1: 使用CLI下载：

1. 注册AWS账号。需要使用信用卡。
2. 下载AWS CLI。下载地址：<https://aws.amazon.com/cn/cli/> 旁边的 "Windows - 下载并运行 64 位 Windows 安装程序。"中进行下载。
3. 在AWS账号管理界面配置用户。**直接使用根用户**（用户权限和用户策略配置非常麻烦）。生成 Access Key ID 和 Secret Access Key 用于数据下载。
4. 使用AWS CLI 中配置用户的Access Key ID 和 Secret Access Key。
   ```cli
   aws configure
   ```
   注意时区设置为数据存储的时区（这好像不必要）。
5. 下载
   1. 下载单个文件
   ```bash
   aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/parquet ./local_folder/
   ```
   2. 下载文件夹
   ```bash
   aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/parquet ./Foursquare/places/ --recursive
   aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/categories/parquet ./Foursquare/categories/ --recursive
   ```

### <a id='toc1_1_3_'></a>[没有验证的方法](#toc0_)

#### <a id='toc1_1_3_1_'></a>[方法2: 使用 boto3 脚本并启用多线程](#toc0_)

AWS 的 Python SDK boto3 支持高效的数据下载。以下是一个多线程下载的示例代码：

```python
import boto3
import os
from concurrent.futures import ThreadPoolExecutor

# 初始化 S3 客户端
s3 = boto3.client('s3', region_name='us-east-1')

def download_file(bucket_name, key, local_path):
    # 创建本地目录
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    # 下载文件
    s3.download_file(bucket_name, key, local_path)
    print(f"Downloaded {key} to {local_path}")

def list_s3_objects(bucket_name, prefix):
    # 列出存储桶中的对象
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
        for obj in page.get('Contents', []):
            yield obj['Key']

# 下载所有文件
bucket = 'fsq-os-places-us-east-1'
prefix = 'release/dt=2024-12-03/places/parquet/'
local_dir = './local_folder/'

with ThreadPoolExecutor(max_workers=10) as executor:
    for key in list_s3_objects(bucket, prefix):
        local_path = os.path.join(local_dir, key[len(prefix):])
        executor.submit(download_file, bucket, key, local_path)
```

说明：
该脚本使用 ThreadPoolExecutor 实现多线程，提高下载速度。
修改 max_workers 值可以调整并行下载的线程数。


#### <a id='toc1_1_3_2_'></a>[方法3: 使用 AWS S3 Transfer Acceleration](#toc0_)

对于跨区域或全球范围内的高速下载，启用 S3 Transfer Acceleration 会显著提高性能。

启用 Transfer Acceleration：

登录到 S3 控制台。
找到目标存储桶，启用 Transfer Acceleration。
注意：需要管理员权限。
使用 CLI 或 boto3 加速下载：


```bash
aws configure set s3.use_accelerate_endpoint true
aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/ ./local_folder/ --recursive
```

#### <a id='toc1_1_3_3_'></a>[方法4: 利用 EC2 实例中转](#toc0_)

如果本地网络速度有限，可以使用 AWS EC2 实例作为中转，下载文件后再传输到本地：

1. 启动 EC2 实例：
   - 在与 S3 存储桶同区域的 AWS 区域中启动 EC2 实例（如 us-east-1）。
2. 配置实例：
    - 确保实例具有正确的 IAM 角色（允许访问目标 S3 存储桶）。
    - 或者在实例中配置 AWS CLI 凭证。
3. 下载数据到 EC2： 使用 AWS CLI 或 boto3 下载数据到实例本地存储。
4. 传输到本地：
   - 使用 SCP 或其他工具将数据从 EC2 下载到本地：
    ```bash
    scp -i your-key.pem ec2-user@your-ec2-ip:/path/to/data ./local_folder/
    ```

#### <a id='toc1_1_3_4_'></a>[最佳实践和优化建议](#toc0_)

1. 优化区域： 确保您所在的区域和存储桶区域一致（us-east-1），以减少网络延迟。
2. 并发下载：
   - 使用 CLI 的并发选项：--parallel（需要 AWS CLI v2 支持）。
   - 或者在自定义脚本中实现多线程或多进程。
3. 减少冗余请求：
   - 使用 --only-show-errors 减少日志。
   - 对于大批量文件，启用 S3 分段下载（boto3 自动支持）。
4. 监控下载性能： 使用 --debug 检查下载性能瓶颈。

## <a id='toc1_2_'></a>[使用](#toc0_)

### <a id='toc1_2_1_'></a>[使用参考](#toc0_)

参考：<https://blog.51cto.com/u_16213329/12389629>

### 数据在 https://huggingface.co/ 上的统计信息

[huggingface上的数据下载及情况说明](https://huggingface.co/datasets/foursquare/fsq-os-places)。

|序号|列名|统计情况|列使用情况|
|---|---|---|---|
|1|fsq_place_id|所有数据都有。|用|
|2|name|所有数据都有。|用|
|3|latitude|null的比例为0.3%。|用|
|4|longitude|null的比例为0.3%。|用|
|5|address|null的比例为35.6%。|用，先用经纬度来填。|
|6|locality|null的比例为27.9%。|用，先用经纬度来填。|
|7|region|null的比例为32.5%。|用，先用经纬度来填。|
|8|postcode|null的比例为45.2%。|丢弃|
|9|admin_region|null的比例为90.1%。|丢弃|
|10|post_town|null的比例为97.4%。|丢弃|
|11|po_box|null的比例为99.7%。|丢弃|
|12|country|所有数据都有。美国占比22.5%。|用|
|13|date_created|null的比例<0.1%。|丢弃|
|14|date_refreshed|null的比例<0.1%。|丢弃|
|15|date_closed|null的比例为94.1%。|丢弃|
|16|tel|null的比例为54.3%。|丢弃|
|17|website|null的比例为69.4%。|丢弃|
|18|email|null的比例为88.8%。|丢弃|
|19|facebook_id|null的比例为93.3%。|丢弃|
|20|instagram|null的比例为98.9%。|丢弃|
|21|twitter|null的比例为95.4%。|丢弃|
|22|fsq_category_ids|null的比例为11.2%。|用|
|23|fsq_category_labels|null的比例为11.2%。|用|
|24|geom|全为空。|丢弃|
|25|bbox|无法统计有效值，里面是一个json。|丢弃。是POI的范围。|


### 通过pandas读取

In [3]:
import pandas as pd

df = pd.read_parquet(r'./Data/Foursquare/places/parquet/places-00000.zstd.parquet')
df.head(5)

,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,tel,website,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,geom,bbox
0,4878c69f3d6c44dbc55b277e,Huge Brands,NaN,NaN,4910 W Amelia Earhart Dr,Salt Lake City,UT,84116,None,None,...,(801) 355-0331,https://www.hugebrands.com,None,NaN,None,None,[52f2ab2ebcbc57f1066b8b28],[Retail > Print Store],None,"{'xmin': None, 'ymin': None, 'xmax': None, 'ym..."
1,aef27cd85bb544efb68fa03a,Альфа,NaN,NaN,"Фрунзе Ул., д. 5",Новосибирск,Новосибирская область,630091,None,None,...,None,http://www.alaba.ru,a@alaba.ru,NaN,None,None,[63be6904847c3692a84b9b98],[Business and Professional Services > Wholesaler],None,"{'xmin': None, 'ymin': None, 'xmax': None, 'ym..."
2,16c1ee7ff3e046d162b771fc,СТИПЛ,NaN,NaN,"Ленинский Пр., 18",Москва,Москва,119071,None,None,...,None,None,None,NaN,None,None,None,None,None,"{'xmin': None, 'ymin': None, 'xmax': None, 'ym..."
3,14f4601b631548f6bb6a9e56,Advance Auto Parts,NaN,NaN,485 US Highway 1,Edison,NJ,08817,None,None,...,(732) 985-3308,https://stores.advanceautoparts.com/nj/edison/...,None,NaN,None,None,[63be6904847c3692a84b9be6],[Retail > Automotive Retail > Car Parts and Ac...,None,"{'xmin': None, 'ymin': None, 'xmax': None, 'ym..."
4,5a26cf2c9de23b0468340907,Ziemasprieki,NaN,NaN,Aroniju Iela 279,Aizkraukle,Aizkraukles novads,5101,None,None,...,None,None,None,NaN,None,None,None,None,None,"{'xmin': None, 'ymin': None, 'xmax': None, 'ym..."


In [2]:
df.columns

Index(['fsq_place_id', 'name', 'latitude', 'longitude', 'address', 'locality',
       'region', 'postcode', 'admin_region', 'post_town', 'po_box', 'country',
       'date_created', 'date_refreshed', 'date_closed', 'tel', 'website',
       'email', 'facebook_id', 'instagram', 'twitter', 'fsq_category_ids',
       'fsq_category_labels', 'geom', 'bbox'],
      dtype='object')

In [4]:
df_category = pd.read_parquet(r'./Data/Foursquare/categories/categories.zstd.parquet')
df_category.head(3)


,category_id,category_level,category_name,category_label,level1_category_id,level1_category_name,level2_category_id,level2_category_name,level3_category_id,level3_category_name,level4_category_id,level4_category_name,level5_category_id,level5_category_name,level6_category_id,level6_category_name
0,63be6904847c3692a84b9c0c,3,Hockey Club,Sports and Recreation > Hockey > Hockey Club,4f4528bc4b90abdf24c9de85,Sports and Recreation,63be6904847c3692a84b9c0b,Hockey,63be6904847c3692a84b9c0c,Hockey Club,None,None,None,None,None,None
1,52960eda3cf9994f4e043acc,5,Indonesian Meatball Restaurant,Dining and Drinking > Restaurant > Asian Resta...,63be6904847c3692a84b9bb5,Dining and Drinking,4d4b7105d754a06374d81259,Restaurant,4bf58dd8d48988d142941735,Asian Restaurant,4deefc054765f83613cdba6f,Indonesian Restaurant,52960eda3cf9994f4e043acc,Indonesian Meatball Restaurant,None,None
2,4bf58dd8d48988d117951735,3,Candy Store,Retail > Food and Beverage Retail > Candy Store,4d4b7105d754a06378d81259,Retail,4bf58dd8d48988d1f9941735,Food and Beverage Retail,4bf58dd8d48988d117951735,Candy Store,None,None,None,None,None,None


### GPT提供的参考代码

代码解析：

- 读取数据：使用 pl.read_csv() 读取数据，适用于大规模数据。
- 处理缺失值：用 fill_null() 替换缺失的数值和类别数据。
- 去重：按 venue_id 去重，避免重复记录影响分析。
- 地理编码：使用 geohash2.encode() 将经纬度转换为 GeoHash，方便后续空间分析。
- 时间特征提取：解析时间戳，并提取 hour（小时）、weekday（星期）。
- 归一化数值特征：用 MinMaxScaler 归一化 rating 和 review_count，提升模型效果。

这段代码适用于 POI 推荐、轨迹分析、时空建模等应用场景。如果你有特定的需求，比如 特定格式的 Foursquare 数据，可以进一步优化。

In [ ]:
import polars as pl
import geohash2
from datetime import datetime

# 读取 Foursquare 数据集（假设格式为 CSV）
df = pl.read_csv("foursquare_poi.csv")

# 展示数据结构
print(df.head())

# 1️⃣ 处理缺失值（填充缺失的类别为空字符串，数值填充为 0）
df = df.fill_null({"category": "", "latitude": 0.0, "longitude": 0.0})

# 2️⃣ 去重
df = df.unique(subset=["venue_id"])

# 3️⃣ 生成地理编码（GeoHash）
def geohash_encode(lat, lon, precision=7):
    return geohash2.encode(lat, lon, precision)

df = df.with_columns(
    pl.struct(["latitude", "longitude"])
    .map_elements(lambda x: geohash_encode(x["latitude"], x["longitude"]), return_dtype=pl.Utf8)
    .alias("geohash")
)

# 4️⃣ 处理时间信息（假设数据有时间戳 'timestamp'）
df = df.with_columns(
    pl.col("timestamp").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S").alias("datetime"),
    pl.col("datetime").dt.hour().alias("hour"),
    pl.col("datetime").dt.weekday().alias("weekday")
)

# 5️⃣ 归一化数值特征（示例：评分、评论数）
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

num_features = ["rating", "review_count"]
df = df.with_columns(
    [pl.Series(name=col, values=scaler.fit_transform(df[col].to_numpy().reshape(-1,1)).flatten()) for col in num_features]
)

# 保存预处理后的数据
df.write_csv("foursquare_poi_preprocessed.csv")

print(df.head())
